In [1]:
%load_ext autoreload
%autoreload 2

import os

current_dir = os.getcwd()
if 'LASO' not in current_dir:
    laso_dir = os.path.join(current_dir, 'LASO')
    if os.path.exists(laso_dir):
        os.chdir(laso_dir)
        print(f"Changed directory to: {laso_dir}")
    else:
        print("LASO directory does not exist in the current path.")
else:
    print(f"Already in LASO directory: {current_dir}")


Changed directory to: /teamspace/studios/this_studio/LASO


In [2]:
import torch
from LASO.model.PointRefer import get_PointRefer

# 加载训练好的模型
model_path = '/teamspace/studios/this_studio/best_model.pt'
model = get_PointRefer()
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
model = model.cuda()
model.eval()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


PointRefer(
  (text_encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [17]:
import numpy as np  # 导入NumPy，用于处理点云数据的数组操作
import plotly.graph_objects as go  # 导入Plotly，用于3D可视化
from torch.utils.data import DataLoader  # 导入DataLoader，用于处理数据批次
from data_utils.shapenetpart import AffordQ  # 导入数据集工具，加载ShapeNet数据

# 加载测试集
test_dataset = AffordQ('train')  # 加载数据集
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2)  # 使用DataLoader按批加载数据，batch_size=1

# 从 test_loader 中抽样一个批次数据
sample = next(iter(test_loader))  # 获取一个批次的数据
points, cls, gt_mask, question, aff_label = sample  # 分解数据，points为点云数据，gt_mask为Ground Truth掩码
points, gt_mask, aff_label = points.cuda(), gt_mask.cuda(), aff_label.cuda()  # 将数据转移到GPU上进行计算

# 对抽样数据进行预测
with torch.no_grad():  # 使用no_grad上下文管理器，以防止计算梯度，减少内存使用
    pred_affordance = model(question, points)  # 使用预训练模型进行推理，预测功能性标签
    pred_affordance = pred_affordance.cpu().numpy()  # 将预测结果从GPU转移回CPU，并转换为NumPy数组

# 将 points 的形状从 [1, 3, 2048] 转换为 [2048, 3]
point_cloud_data = points[0].transpose(0, 1).cpu().numpy()  # 调整点云数据的形状，并从GPU转移到CPU

# 给 Ground Truth 点云上色
gt_colors = np.zeros((point_cloud_data.shape[0], 3))  # 创建一个与点云点数相同的颜色数组，初始化为全零
reference_color = np.array([255, 0, 0])  # 定义参考颜色为红色
back_color = np.array([190, 190, 190])  # 定义背景颜色为灰色
for i, point_affordance in enumerate(gt_mask[0].cpu().numpy()):  # 遍历每个点的功能性标签
    scale_i = point_affordance  # 取该点的功能性值
    gt_colors[i] = (reference_color - back_color) * scale_i + back_color  # 根据功能性值将颜色从背景色过渡到参考色
gt_colors = gt_colors / 255.0  # 将颜色值归一化到 [0, 1] 范围内，符合RGB标准

# 给预测结果点云上色
pred_colors = np.zeros((point_cloud_data.shape[0], 3))  # 创建与Ground Truth类似的颜色数组用于预测结果
for i, aff_pred in enumerate(pred_affordance[0]):  # 遍历每个点的预测功能性标签
    scale_i = aff_pred  # 取该点的预测功能性值
    pred_colors[i] = (reference_color - back_color) * scale_i + back_color  # 同样地，根据功能性值为点云上色
pred_colors = pred_colors / 255.0  # 将预测结果的颜色值归一化到 [0, 1]

# 创建 Plotly 图形，用于展示 Ground Truth 点云
fig = go.Figure()  # 创建一个空的Plotly Figure对象

# Ground Truth 点云
fig.add_trace(go.Scatter3d(
    x=point_cloud_data[:, 0],  # X 轴坐标，表示水平宽度
    y=point_cloud_data[:, 2],  # Z 轴坐标，表示水平深度
    z=point_cloud_data[:, 1],  # Y 轴坐标，表示垂直高度
    mode='markers',  # 以点的形式展示点云
    marker=dict(
        size=3,  # 点的大小
        color=['rgb({}, {}, {})'.format(r*255, g*255, b*255) for r, g, b in gt_colors],  # 使用 Ground Truth 颜色
        opacity=0.8  # 设置点的透明度
    ),
    name='Ground Truth'  # 图例名称
))

# 预测结果点云
fig.add_trace(go.Scatter3d(
    x=point_cloud_data[:, 0],  # X 轴坐标
    y=point_cloud_data[:, 2],  # Z 轴坐标
    z=point_cloud_data[:, 1],  # Y 轴坐标
    mode='markers',  # 以点的形式展示点云
    marker=dict(
        size=3,  # 点的大小
        color=['rgb({}, {}, {})'.format(r*255, g*255, b*255) for r, g, b in pred_colors],  # 使用预测结果的颜色
        opacity=0.8  # 设置点的透明度
    ),
    name='Prediction'  # 图例名称
))

# 设置图形布局，使用 XZ 作为水平面，Y 为高度
fig.update_layout(
    title="Point Cloud Visualization (Ground Truth vs Prediction, XZ as Horizontal)",  # 设置图形标题
    scene=dict(
        xaxis_title="X",  # X 轴标签
        yaxis_title="Z (Horizontal)",  # Z 轴标签作为水平面
        zaxis_title="Y (Height)",  # Y 轴标签表示高度
        aspectmode="cube"  # 保持 X、Y、Z 轴的比例相同，使点云不会被拉伸或压缩
    ),
    margin=dict(l=0, r=0, b=0, t=0)  # 设置图形的边距，四边距设为0，使图形占满显示区域
)

# 将 question 转换为字符串形式
question_text = question[0]

# 设置标题，包含 question 信息
fig.update_layout(
    title=f"Question: {question_text}",  # 添加 question
    scene=dict(
        xaxis_title="X",  # X 轴标签
        yaxis_title="Z (Horizontal)",  # Z 轴标签作为水平面
        zaxis_title="Y (Height)",  # Y 轴标签表示高度
        aspectmode="cube"  # 保持 X、Y、Z 轴的比例相同，使点云不会被拉伸或压缩
    ),
    margin=dict(l=0, r=0, b=0, t=50)  # 设置图形边距
)

# 显示图形
fig.show()  # 在当前环境中显示图形
# fig.write_html("point_cloud_visualization.html") # 如果你需要保存这个可视化为一个html文件，可取消这行注释


load train set successfully, lenth 16120
